In [17]:
(
exec 9<&1

echopat='*echo $?*'

# The code below seems to work, but it may be sensitive to
# the timeout and sleep values.  Hopefully it works well enough
# until we have time to modify the bash_kernel code.

blockingread()
{
   while sleep 0.2; do
      IFS= read -r ln
      if [[ "$ln" == $echopat ]]; then
         # give Jupyter a return code when it asks for one
         echo "0" >&9
         echo -n "$PS1" >&9
         continue
      fi
      [ "$ln" = "" ] && continue
      break
   done
}

readcell() {
    blockingread
    echo "$ln"
    echo -n "$PS2">&9
    # read rest of the cell contents
    while IFS= read -t 0.2 -r lnr; do
      ## somehow get make this next line unnecessary!!!
      [[ "$lnr" == *endofcell* ]] && break
      echo "$lnr"
      # The problem is this next line.  Jupyter will not send the next line unless
      # PS2 is sent.  But if there are no more lines, then Jupyter will block.
      # Therefore, it is necessary to modify the bash_kernel code to remove the
      # requirement for the endofcell marker.
      echo -n "$PS2">&9
    done
}
   
echo "Ready."
echo -n "$PS1"

# for testing:
while sleep 0.2; do
    echo $$ >>/tmp/sbash.log
    ls -l /proc/$BASHPID/fd >>/tmp/sbash.log
    input="$(readcell)"
    if [[ "$input" = *quit* ]]; then
      echo "bye"
      # the PS1 will be sent by the parent bash (the one created by jupyter)
      break
    fi
    echo "begin test echo of cell contents:"
    echo "$input"
    echo "end"
    echo -n "$PS1"
done
)

Ready.


In [10]:
echo \
  a \
  b
  endofcell

begin test echo of cell contents:
echo \
  a \
  b
end


In [11]:
line one
line two
endofcell

begin test echo of cell contents:
line one
line two
end


In [4]:
one
endofcell

begin test echo of cell contents:
one
end


In [18]:
quit
endofcell

bye


In [1]:
export PS1 ; export PS2 ; while read -r ln ; do   echo "$ln" ; sleep 1 ; done | tee -a /tmp/ii | script /tmp/null | tee -a /tmp/oo

In [16]:
date
whoami

centos
